In [1]:
import os
import numpy as np
import torch
import torch_geometric.data

import meld_graph
import meld_graph.training
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
from meld_graph.paths import load_config
from meld_graph.paths import EXPERIMENT_PATH
from meld_graph.dataset import GraphDataset, Oversampler

Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/
Setting FS_SUBJECTS_PATH to 
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1


In [2]:
# create experiment
config_file = '/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/scripts/config_files/base_config_test.py'
config = load_config(config_file)
exp = meld_graph.experiment.Experiment(config.network_parameters, config.data_parameters)


Saving parameter files to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1/23-05-19_MATH_test_histology_head/fold_00
Initialised Experiment 23-05-19_MATH_test_histology_head


In [3]:
import importlib
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.data_preprocessing)
importlib.reload(meld_graph.training)

<module 'meld_graph.training' from '/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/meld_graph/training.py'>

In [4]:
# create the trainer the model
trainer = meld_graph.training.Trainer(exp)
trainer.experiment.load_model()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainer.experiment.model.to(device)
# trainer.train()


Creating model


In [10]:
train_dset = GraphDataset.from_experiment(trainer.experiment, mode='train')
print(train_dset)


Getting train val test split
total number of subjects: 45
total number of subjects after restricting to subjects from MELD_dataset_V6.csv: 45
total number of subjects: 950
total number of subjects after restricting to subjects from MELD_dataset_V6.csv: 942
total number after filtering by scanner ['3T', '15T'], features, lesional_only True: 911
full_feature_list: ['.combat.on_lh.curv.sm5.mgh', '.combat.on_lh.gm_FLAIR_0.25.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.5.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.75.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.sm10.mgh', '.combat.on_lh.pial.K_filtered.sm20.mgh', '.combat.on_lh.sulc.sm5.mgh', '.combat.on_lh.thickness.sm10.mgh', '.combat.on_lh.w-g.pct.sm10.mgh', '.combat.on_lh.wm_FLAIR_0.5.sm10.mgh', '.combat.on_lh.wm_FLAIR_1.sm10.mgh', '.inter_z.asym.intra_z.combat.on_lh.curv.sm5.mgh', '.inter_z.asym.intra_z.combat.on_lh.gm_FLAIR_0.25.sm10.mgh', '.inter_z.asym.intra_z.combat.on_lh.gm_FLAIR_0.5.sm10.mgh', '.inter_z.asym.intra_z.combat.on_lh.gm_FLAIR_0.75.sm10.mgh

GraphDataset(38)


In [11]:
val_dset = GraphDataset.from_experiment(trainer.experiment, mode='val')
print(val_dset)

Loading and preprocessing val data
Z-scoring data for MELD_H10_3T_C_0007
Z-scoring data for MELD_H10_3T_C_0002
Z-scoring data for MELD_H10_3T_FCD_0003
Z-scoring data for MELD_H10_3T_C_0030


GraphDataset(8)


In [22]:
sampler=None
shuffle = trainer.params['shuffle_each_epoch']
batch_size = trainer.params['batch_size']
optimiser = torch.optim.SGD(trainer.experiment.model.parameters(), **trainer.params['optimiser_parameters'])
optimiser = optimiser

train_data_loader = torch_geometric.loader.DataLoader(
             train_dset, sampler=sampler, 
             shuffle=shuffle,
             batch_size= batch_size,
             num_workers=0, persistent_workers=False, prefetch_factor=2
             )

In [14]:
cur_scores = trainer.train_epoch(train_data_loader, optimiser)

/home/co-ripa1/.conda/envs/meld_graph/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/co-ripa1/.conda/envs/meld_graph/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/meld_graph/training.py:449: RuntimeWarning: invalid value encountered in long_scalars
  elif metric == 'histo_precision':


In [10]:
cur_scores

{'cross_entropy': 0.6554559230804443,
 'dice': 0.9847615841776133,
 'distance_regression': 0.4588208019733429,
 'lesion_classification': 0.745412266254425,
 'histology_classification': 1.4951695680618287,
 'ds6_cross_entropy': 0.042364323139190675,
 'ds5_cross_entropy': 0.059493355453014374,
 'ds4_cross_entropy': 0.1940973311662674,
 'ds3_cross_entropy': 0.4693894386291504,
 'ds6_dice': 0.061441260296851395,
 'ds5_dice': 0.12169606997631491,
 'ds4_dice': 0.24401496760547162,
 'ds3_dice': 0.4835611624643207,
 'ds6_distance_regression': 0.025491341203451156,
 'ds5_distance_regression': 0.039177677035331725,
 'ds4_distance_regression': 0.13558775037527085,
 'ds3_distance_regression': 0.27978743612766266,
 'ds6_lesion_classification': nan,
 'ds5_lesion_classification': nan,
 'ds4_lesion_classification': nan,
 'ds3_lesion_classification': nan,
 'ds6_histology_classification': nan,
 'ds5_histology_classification': nan,
 'ds4_histology_classification': nan,
 'ds3_histology_classification': na

In [5]:
trainer.train(wandb_logging=False)

Model already exists. Specify force=True to force reloading and initialisation
Creating model
Getting train val test split
total number of subjects: 45
total number of subjects after restricting to subjects from MELD_dataset_V6.csv: 45
total number of subjects: 950
total number of subjects after restricting to subjects from MELD_dataset_V6.csv: 942
total number after filtering by scanner ['3T', '15T'], features, lesional_only True: 911
full_feature_list: ['.combat.on_lh.curv.sm5.mgh', '.combat.on_lh.gm_FLAIR_0.25.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.5.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.75.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.sm10.mgh', '.combat.on_lh.pial.K_filtered.sm20.mgh', '.combat.on_lh.sulc.sm5.mgh', '.combat.on_lh.thickness.sm10.mgh', '.combat.on_lh.w-g.pct.sm10.mgh', '.combat.on_lh.wm_FLAIR_0.5.sm10.mgh', '.combat.on_lh.wm_FLAIR_1.sm10.mgh', '.inter_z.asym.intra_z.combat.on_lh.curv.sm5.mgh', '.inter_z.asym.intra_z.combat.on_lh.gm_FLAIR_0.25.sm10.mgh', '.inter_z.asym.intra_z.c

KeyboardInterrupt: 